Let's load pandas:

In [ ]:
import pandas as pd

### Loading data

Load some traffic accident data directly from the web ([NYC Crash Mapper](http://crashmapper.org/#/?cfat=true&cinj=true&endDate=2019-03&geo=citywide&identifier=&lngLats=%255B%255D&mfat=true&minj=true&noInjFat=false&pfat=true&pinj=true&startDate=2018-03)):

In [ ]:
url="https://chekpeds.carto.com/api/v2/sql?q=SELECT%20c.cartodb_id%2C%20c.socrata_id%2C%20c.the_geom%2C%20c.on_street_name%2C%20c.cross_street_name%2C%20c.date_val%20AS%20date_time%2C%20c.latitude%2C%20c.longitude%2C%20c.borough%2C%20c.zip_code%2C%20c.crash_count%2C%20c.number_of_cyclist_injured%2C%20c.number_of_cyclist_killed%2C%20c.number_of_motorist_injured%2C%20c.number_of_motorist_killed%2C%20c.number_of_pedestrian_injured%2C%20c.number_of_pedestrian_killed%2C%20c.number_of_persons_injured%2C%20c.number_of_persons_killed%2C%20array_to_string(c.contributing_factor%2C%20%27%2C%27)%20as%20contributing_factors%2C%20array_to_string(c.vehicle_type%2C%20%27%2C%27)%20as%20vehicle_types%20FROM%20crashes_all_prod%20c%20%20WHERE%20(%20year%3A%3Atext%20%7C%7C%20LPAD(month%3A%3Atext%2C%202%2C%20%270%27)%20%3C%3D%20%272019%27%20%7C%7C%20LPAD(3%3A%3Atext%2C%202%2C%20%270%27)%20)%20AND%20(%20year%3A%3Atext%20%7C%7C%20LPAD(month%3A%3Atext%2C%202%2C%20%270%27)%20%3E%3D%20%272018%27%20%7C%7C%20LPAD(3%3A%3Atext%2C%202%2C%20%270%27)%20)%20%20AND%20(%20number_of_cyclist_injured%20%3E%200%20OR%20number_of_motorist_injured%20%3E%200%20OR%20number_of_pedestrian_injured%20%3E%200%20%20OR%20%20number_of_cyclist_killed%20%3E%200%20OR%20number_of_motorist_killed%20%3E%200%20OR%20number_of_pedestrian_killed%20%3E%200%20)&format=CSV"
accidents = pd.read_csv(url)

Let's take a glimpse at the dataframe:

In [ ]:
accidents

Loading a local file is just as easy:

In [ ]:
accidents = pd.read_csv("accidents.csv") # <- this is actually the same dataset we loaded from the web before
accidents

Let's check out the data types on our accidents data

In [ ]:
accidents.dtypes

Strings and the timestamp are stored as _objects_ – we'll fix that in a bit.

Let's try to access parts of the accidents DataFrame. First, a single column. This will return a pandas [Series](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.html).

In [ ]:
accidents['vehicle_types']

Slicing rows – returns a new DataFrame:

In [ ]:
accidents[2:10] # rows 2 to 9

In [ ]:
accidents[2:]   # all rows from row 2

In [ ]:
accidents[:10]  # all rows up to row 9

### Using .loc

In [ ]:
accidents.loc[3]

Using .loc to pick columns:

In [ ]:
accidents.loc[27:29, ['number_of_cyclist_injured', 'number_of_cyclist_killed']]

### Using .iloc

... to get the same result.

In [ ]:
accidents.iloc[27:30, 11:13]

### DataFrame indexes

In [ ]:
accidents.index

In [ ]:
accidents.columns

In [ ]:
accidents.set_index(pd.to_datetime(accidents['date_time']), inplace=True)
accidents.set_index(accidents.index.tz_localize(None), inplace=True)
accidents.index

Now we can easily query the accidents by date range:

In [ ]:
accidents.loc['2019-01-01':'2019-02-01']

How many accidents do we have per month of the year?

In [ ]:
accidents.resample('M').sum()

..and per hour?

In [ ]:
accidents.resample('H').sum()  

How many accidents are there on average per hour of the day?

In [ ]:
hourly_accidents = accidents.resample('H').sum()
hourly_accidents['hour'] = hourly_accidents.index.hour
hourly_accidents.groupby(['hour']).mean()

Let's plot this:

In [ ]:
%matplotlib inline # you only need this one once – all plots from here on will be embedded in the notebook

mean_hourly_accidents = hourly_accidents.groupby(['hour']).mean()
mean_hourly_accidents.plot(y='crash_count')

# 🏋 Exercise

**Which month of the year has the highest number of accidents?**

In [ ]:
# your code here...

### Boolean indexing

In [ ]:
accidents['number_of_cyclist_injured'] > 0

Filter the dataframe using this boolean series:

In [ ]:
accidents[accidents['number_of_cyclist_injured'] > 0]

Combine multiple citeria:

In [ ]:
accidents[(accidents['number_of_cyclist_injured'] > 0) | (accidents['number_of_cyclist_killed'] > 0)]

Query using string matching:

In [ ]:
accidents[accidents['vehicle_types'].str.contains('Taxi')]

Do we have null values in our ```vehicle_types``` column?

In [ ]:
accidents[accidents['vehicle_types'].isnull()]

Let's replace the NaN values with an empty string:

In [ ]:
accidents.vehicle_types.fillna('', inplace=True)
accidents[accidents['vehicle_types'].isnull()]

Now this should work:

In [ ]:
accidents[accidents['vehicle_types'].str.contains('Taxi')]

Writing the filtered dataset to a CSV file:

In [ ]:
taxi_accidents = accidents[accidents['vehicle_types'].str.contains('Taxi')]
taxi_accidents.to_csv('taxi_accidents.csv')

### Plotting and (simple) mapping

In [ ]:
accidents.plot(x='longitude', 
               y='latitude', 
               kind='scatter')

Remove the outliers:

In [ ]:
accidents = accidents[ (accidents['latitude'] > 30) & (accidents['longitude'] > -100)  ]
accidents.plot.scatter(x='longitude', 
               y='latitude')

Let's make a fancy map. See the docs for explanation of the [variables](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.plot.html#pandas.DataFrame.plot) and [color scales](https://matplotlib.org/users/colormaps.html).

In [ ]:
accidents[accidents['number_of_persons_injured'] > 0].plot.scatter(x='longitude', 
               y='latitude', 
               c='number_of_persons_injured',
               s=0.7,
               title='NYC accidents with injuries',                                                   
               colormap='Wistia',
               figsize=[14,14])

Let's take a look at hexbins to identify potential hotspots:

In [ ]:
accidents.plot.hexbin(x='longitude', 
               y='latitude',
               figsize=[14,14])

A KDE map using Seaborn:

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sns.kdeplot(accidents['longitude'], 
            accidents['latitude'],
            cmap='Reds')
plt.savefig('kde.pdf') # this one has to be in the same cell that produces the plot!